In [1]:
import numpy as np
import pandas as pd
from scipy import stats

from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification, make_regression
from sklearn.metrics import roc_auc_score
import shap
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier

from hyperopt import hp
from hyperopt import Trials

from xgboost import *

try:
    from shaphypetune import BoostSearch, BoostBoruta, BoostRFE, BoostRFA
except:
    !pip install --upgrade shap-hypetune
    from shaphypetune import BoostSearch, BoostBoruta, BoostRFE, BoostRFA

import warnings
warnings.simplefilter('ignore')

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.cluster.hierarchy as sch
import importlib
import s3finance as s3c
from labellines import labelLines
from datetime import date, datetime, timedelta
from dateutil.relativedelta import relativedelta
from tqdm import tqdm
from functools import partialmethod
from category_encoders import TargetEncoder
import seaborn as sns
from matplotlib import pyplot as plt, dates
from sklearn.metrics import confusion_matrix, auc, roc_curve
from statistics import mean
import pandas_market_calendars as mcal
import joblib

tqdm.__init__ = partialmethod(tqdm.__init__, disable=True)

In [3]:
def binary_performances(y_true, y_prob, thresh=0.5, labels=['Positives','Negatives']):
    
    shape = y_prob.shape
    if len(shape) > 1:
        if shape[1] > 2:
            raise ValueError('A binary class problem is required')
        else:
            y_prob = y_prob[:,1]
    
    plt.figure(figsize=[15,4])
    
    #1 -- Confusion matrix
    cm = confusion_matrix(y_true, (y_prob>thresh).astype(int))
    
    plt.subplot(131)
    ax = sns.heatmap(cm, annot=True, cmap='Blues', cbar=False, 
                     annot_kws={"size": 14}, fmt='g')
    cmlabels = ['True Negatives', 'False Positives',
               'False Negatives', 'True Positives']
    for i,t in enumerate(ax.texts):
        t.set_text(t.get_text() + "\n" + cmlabels[i])
    plt.title('Confusion Matrix', size=15)
    plt.xlabel('Predicted Values', size=13)
    plt.ylabel('True Values', size=13)
      
    #2 -- Distributions of Predicted Probabilities of both classes
    plt.subplot(132)
    plt.hist(y_prob[y_true==1], density=True, bins=25,
             alpha=.5, color='green',  label=labels[0])
    plt.hist(y_prob[y_true==0], density=True, bins=25,
             alpha=.5, color='red', label=labels[1])
    plt.axvline(thresh, color='blue', linestyle='--', label='Boundary')
    plt.xlim([0,1])
    plt.title('Distributions of Predictions', size=15)
    plt.xlabel('Positive Probability (predicted)', size=13)
    plt.ylabel('Samples (normalized scale)', size=13)
    plt.legend(loc="upper right")
    
    #3 -- ROC curve with annotated decision point
    fp_rates, tp_rates, _ = roc_curve(y_true, y_prob)
    roc_auc = auc(fp_rates, tp_rates)
    plt.subplot(133)
    plt.plot(fp_rates, tp_rates, color='orange',
             lw=1, label='ROC curve (area = %0.3f)' % roc_auc)
    plt.plot([0, 1], [0, 1], lw=1, linestyle='--', color='grey')
    tn, fp, fn, tp = [i for i in cm.ravel()]
    plt.plot(fp/(fp+tn), tp/(tp+fn), 'bo', markersize=8, label='Decision Point')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate', size=13)
    plt.ylabel('True Positive Rate', size=13)
    plt.title('ROC Curve', size=15)
    plt.legend(loc="lower right")
    plt.subplots_adjust(wspace=.3)
    plt.show()

    tn, fp, fn, tp = [i for i in cm.ravel()]
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    F1 = 2*(precision * recall) / (precision + recall)
    results = {
        "Precision": precision, "Recall": recall,
        "F1 Score": F1, "AUC": roc_auc
    }
    
    prints = [f"{kpi}: {round(score, 3)}" for kpi,score in results.items()]
    prints = ' | '.join(prints)
    print(prints)
    
    return results

def AUC(y_true, y_hat):
    return 'auc', roc_auc_score(y_true, y_hat), True

In [4]:
# set up access to access class
#
AWS_BUCKET = "w210-wrds-data"
importlib.reload(s3c)
s3f = s3c.s3finance(AWS_BUCKET)

In [6]:
#
# Read all Price Data
#
# get all the financial ratios
all_ratios = s3f.getsummaryallratiopricesfinancial()

In [7]:
# get all the stock prices
all_prices = s3f.getsummaryprices()
#all_prices = None

In [8]:
# get SPY prices
s3f = s3c.s3finance(AWS_BUCKET,version='1658134425.075057')
spyprices=s3f.getstockprices("SPY")

In [9]:
from sklearn.model_selection import RandomizedSearchCV

param_grid = {
    'learning_rate': [0.01, 0.05, 0.1,0.2],
    'num_leaves': [25, 35,100],
    'n_estimators': [100,500,1000,2000],
    'max_depth':  [3, 5, 6, 10, 15, 20],
    'colsample_bytree': [0.3, 0.7, 1.0]
}

params_random = { 'max_depth': [3, 5, 6, 10, 15, 20],
           'learning_rate': [0.01, 0.1, 0.2, 0.3],
           'subsample': np.arange(0.5, 1.0, 0.1),
           'colsample_bytree': np.arange(0.4, 1.0, 0.1),
           'colsample_bylevel': np.arange(0.4, 1.0, 0.1),
           'n_estimators': [100, 500, 1000]}

param_dist = {
    'learning_rate': stats.uniform(0.01, 0.25),
    'colsample_bytree': np.arange(0.4, 1.0, 0.1),
    'colsample_bylevel': np.arange(0.4, 1.0, 0.1),
#    'num_leaves': stats.randint(20,100),
    'n_estimators': [100,500,1000,2000],
    'max_depth': [3, 5, 6, 10]
}

param_dist_hyperopt = {
    'max_depth': 15 + hp.randint('num_leaves', 5), 
    'learning_rate': hp.loguniform('learning_rate', np.log(0.01), np.log(0.2)),
    'colsample_bytree': hp.uniform('colsample_by_tree', 0.6, 1.0)
}



In [11]:
# predict for individual stock
#
def runcyclesticker(all_ratios,ncycles=12,period=10,step=1, years_ahead = 1, predict_start=None,train_start='2000-01-01',end_period='2021-01-01', ticker=[], 
                    c1=None, min_price=5, market_cap=10000,output_analysis=True,model_data_filename="model-period-{period}-year-{cycles}-cycles", savemodel_cycle=12):
    
    if predict_start is None:
        predict_start = (pd.to_datetime(train_start) + relativedelta(years=(period+years_ahead))).strftime("%Y-%m-%d")  
 
    #print(predict_start)
    start_date_predict = predict_start
    start_date_train = train_start
    end_date = end_period
    train_period = period # number of years
    #+ relativedelta(years=1)
    
    tpredict = np.arange(start_date_predict, end_date,np.timedelta64(step, 'M'),  dtype='datetime64[M]').astype('datetime64[D]').tolist() 
    ttrain = np.arange(start_date_train, end_date,np.timedelta64(step, 'M'),  dtype='datetime64[M]').astype('datetime64[D]').tolist()
    listcycles = list(zip(range(ncycles),tpredict,ttrain))

    results = pd.DataFrame(data=[],columns=['date'] + c1 + ['score'],index = list(range(len(listcycles))))
    shaplist = []
    models = []
    for i, p,t in tqdm(listcycles,disable=True):
        train_set = all_ratios[(all_ratios['monthly_date'] >= t) & 
                               (all_ratios['monthly_date'] < (t+ relativedelta(years=train_period))) & (all_ratios['prccd'] >= min_price) & (all_ratios['market_cap'] >= market_cap)]
        encoder = TargetEncoder()
        train_set['stock_encoder'] = encoder.fit_transform(train_set['cusip'].astype(str), train_set['outperformed'])
        if len(ticker)> 0:
            test_set=all_ratios[(all_ratios['monthly_date'] >= p) & (all_ratios['monthly_date'] < (p+ relativedelta(months=step))) & 
                                   (all_ratios['ticker'].isin(ticker)) & (all_ratios['prccd'] >= min_price) & (all_ratios['market_cap'] >= market_cap)]
        else:
            test_set=all_ratios[(all_ratios['monthly_date'] >= p) & 
                                (all_ratios['monthly_date'] < (p+ relativedelta(months=step))) 
                                & (all_ratios['prccd'] >= min_price) & (all_ratios['market_cap'] >= market_cap)]
        # indices_to_keep = ~test_set.isin([np.nan, np.inf, -np.inf]).any(1)
        # test_set = test_set[indices_to_keep]
        encoder = TargetEncoder()
        test_set['stock_encoder'] = encoder.fit_transform(test_set['cusip'].astype(str), test_set['outperformed'])
        X = train_set[c1]
        y = train_set['outperformed']
        Xt = test_set[c1]
        yt = test_set['outperformed']
        print(len(X),len(y),len(Xt),len(yt))
        # clf_xgb2 = XGBClassifier(n_estimators=1000, random_state=0, verbosity=0, n_jobs=-1,learning_rate =0.1,scoring='roc_auc') #n_jobs=-1)
        clf_xgb2 = XGBClassifier(learning_rate =0.1, n_estimators=1000, max_depth=10, min_child_weight=1, gamma=0.3, subsample=0.8,
                                 colsample_bytree=0.8, objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27,n_jobs=-1,
                                 #scoring='roc_auc'
                                )
        model2 = RandomizedSearchCV(estimator=clf_xgb2,
                                    param_distributions=param_dist,
                                    scoring='accuracy',
                                    # scoring='f1',
                                    # scoring='roc_auc',
                                    n_iter=50,
                                    verbose=0,n_jobs=-1)
        # model2 = BoostBoruta(
        #     clf_xgb2, param_grid=param_grid, max_iter=200, perc=100,
        #     importance_type='shap_importances', train_importance=False)
        #model2 = BoostBoruta(clf_xgb2, max_iter=200, perc=100, n_jobs=-1)
        # model2 = clf_xgb2 
        model2.fit(X, y, eval_set=[(Xt, yt)], early_stopping_rounds=6, verbose=0) #  
        #plt.barh(c1, model2.feature_importances_)
        # Fits the explainer
        explainer = shap.Explainer(model2.predict, Xt,seed=42)
        # Calculates the SHAP values - It takes some time
        shap_values = explainer(Xt)
        shaplist.append(shap_values)
        results.iloc[i] = [p] + list(shap_values[0].values) + [model2.score(Xt, yt)]
        if output_analysis:
            binary_performances(yt, model2.predict_proba(Xt),thresh=0.5) # thresh = 0.5)
            shap.plots.bar(shap_values,max_display=50)
            shap.summary_plot(shap_values,Xt,max_display=50)
        models.append([model2,Xt,yt,test_set])
        if ((i + 1) % savemodel_cycle) == 0:
            joblib.dump([results,shaplist,models], f"model-period-{period}-year-{i+1}-cycles.pkl")
        #print(results)
        
    return results,shaplist,models

In [12]:
keyfields = [x for x in list(all_ratios.select_dtypes(include=['number']).columns) if x not in \
             ["one_yr_chg", "one_yr_chg_spy", "outperformed",'prccd', 'prchd', 'prcld', 'prcod', 'prcstd']]

In [13]:
%%time
results7,shaplist7,models7 = runcyclesticker(all_ratios, ncycles=120,period=7,step=1,train_start='2008-01-01', c1=list(set(keyfields)), 
                                          min_price=5, market_cap=300,output_analysis=False) # 2 year 2011, 3 year 2010, 5 year 2008, 7 year 2006, 10 year 2003
#results,shaplist,models = runcyclesticker(all_ratios, ncycles=12,period=7,step=1,train_start='2004-01-01', c1=list(set(keyfields)))

155420 155420 2003 2003


Permutation explainer: 2004it [02:09, 14.56it/s]                                                                                                                                                                          


155551 155551 2018 2018


Permutation explainer: 2019it [01:56, 16.06it/s]                                                                                                                                                                          


155771 155771 2030 2030


...

173558 173558 2295 2295


Permutation explainer: 2296it [01:25, 23.37it/s]                                                                                                                                                                          


173831 173831 2333 2333


Permutation explainer: 2334it [01:14, 26.98it/s]                                                                                                                                                                          


CPU times: user 3d 9h 15min 2s, sys: 35min 8s, total: 3d 9h 50min 10s
Wall time: 19h 15min 42s


In [14]:
%%time
results5,shaplist5,models5 = runcyclesticker(all_ratios, ncycles=120,period=5,step=1,train_start='2008-01-01', c1=list(set(keyfields)), 
                                          min_price=5, market_cap=300,output_analysis=False)

107179 107179 2034 2034


Permutation explainer: 2035it [02:31, 12.41it/s]                                                                                                                                                                          


107110 107110 2072 2072


Permutation explainer: 2073it [02:22, 12.76it/s]                                                                                                                                                                          


107082 107082 2059 2059


...

125829 125829 2295 2295


Permutation explainer: 2296it [02:03, 17.03it/s]                                                                                                                                                                          


125846 125846 2333 2333


Permutation explainer: 2334it [01:43, 20.24it/s]                                                                                                                                                                          


CPU times: user 2d 21h 25min 14s, sys: 33min 36s, total: 2d 21h 58min 51s
Wall time: 16h 10min 37s


In [15]:
%%time
results3,shaplist3,models3 = runcyclesticker(all_ratios, ncycles=120,period=3,step=1,train_start='2008-01-01', c1=list(set(keyfields)), 
                                          min_price=5, market_cap=300,output_analysis=False)

62905 62905 1837 1837


Permutation explainer: 1838it [01:07, 23.66it/s]                                                                                                                                                                          


62832 62832 1844 1844


Permutation explainer: 1845it [01:16, 20.95it/s]                                                                                                                                                                          


62838 62838 1851 1851


...

75879 75879 2295 2295


Permutation explainer: 2296it [01:05, 29.65it/s]                                                                                                                                                                          


75895 75895 2333 2333


Permutation explainer: 2334it [01:06, 29.85it/s]                                                                                                                                                                          


CPU times: user 2d 3h 18min, sys: 23min 24s, total: 2d 3h 41min 24s
Wall time: 11h 36min 30s


In [18]:
%%time
results10,shaplist10,models10 = runcyclesticker(all_ratios, ncycles=180,period=10,step=1,train_start='2000-01-01', c1=list(set(keyfields)), 
                                          min_price=5, market_cap=300,output_analysis=False)

221930 221930 1864 1864


Permutation explainer: 1865it [00:55, 27.37it/s]                                                                                                                                                                          


221830 221830 1908 1908


Permutation explainer: 1909it [00:57, 27.61it/s]                                                                                                                                                                          


221666 221666 1908 1908


...

239303 239303 2295 2295


Permutation explainer: 2296it [01:46, 19.47it/s]                                                                                                                                                                          


239700 239700 2333 2333


Permutation explainer: 2334it [01:19, 25.56it/s]                                                                                                                                                                          


CPU times: user 7d 18h 48min 20s, sys: 1h 18min 25s, total: 7d 20h 6min 45s
Wall time: 1d 18h 50min 58s


In [19]:
resultsf = results7.loc[:, results.columns !='date'].astype(float)
results1 = resultsf.loc[:, (resultsf != 0.0).any(axis=0)]
results1['date'] = results7['date'].astype(str)
f = plt.figure()
ax = results1.set_index('date').plot(use_index=True, rot=90,figsize=(30, 20),legend=True,ax=f.gca())
foo=labelLines(ax.get_lines(),  align=False,  color="k")
plt.legend(loc='center left', bbox_to_anchor=(1.0, 0.5))
plt.show()

NameError: name 'results' is not defined

In [23]:
len(results5)#joblib.dump([results10,shaplist10,models10], f"model-period-{10}-year-{120}-cycles.pkl")


84

In [ ]:
# filename = 'model-period-7-year-12-cycles_exp_1.sav'
# joblib.dump([results,shaplist,models], filename)

In [ ]:
#outputhomepage,stockpicks = drawallstockinfoannual(spyprices,all_ratios,all_prices,models,numstocks=1,show_all_stocks=True,months=1)

In [ ]:
#outputhomepage1,stockpicks1 = drawallstockinfoannual(spyprices,all_ratios,all_prices,loaded_models,numstocks=5,show_all_stocks=False)

In [ ]:
# import tempfile
# import boto3
# import joblib

# s3_client = boto3.client('s3')
# bucket_name = "my-bucket"
# key = "model.pkl"

# # WRITE
# with tempfile.TemporaryFile() as fp:
#     joblib.dump(model, fp)
#     fp.seek(0)
#     s3_client.put_object(Body=fp.read(), Bucket=bucket_name, Key=key)

# # READ
# with tempfile.TemporaryFile() as fp:
#     s3_client.download_fileobj(Fileobj=fp, Bucket=bucket_name, Key=key)
#     fp.seek(0)
#     model = joblib.load(fp)

# # DELETE
# s3_client.delete_object(Bucket=bucket_name, Key=key)